In [ ]:
import numpy as np

Hands-on with the ring test model
=================================

This notebook is meant to provide the building blocks for exploring the performance impacts of various NEURON and CoreNEURON options, using the ring test model (`ringtest.py`).

This model uses a custom MOD file (`halfgap.mod`), so we must start by building `special` using this:

In [ ]:
!nrnivmodl mod

Now we can run the `ringtest.py` script, passing any options we want to:

In [ ]:
!x86_64/special -python ringtest.py -nt 1

The above command executed in around 0.2s on the author's machine.
**Is that good?** *The author has no idea...*

Typically when examining the performance of a new model, or an existing model on a new system, or indeed a change in software version, we need to look at trends and comparisons.

To illustrate this, we will run the same model using different numbers of CPU threads.
This is steered by the `-nt` option to `ringtest.py`.

In [ ]:
def ringtest(*args, repeat=3):
    """TODO: update ringtest.py to write these somewhere and avoid regexing"""
    import re
    from subprocess import check_output, STDOUT

    def run():
        out = check_output(
            ["./x86_64/special", "-python", "ringtest.py"] + [str(x) for x in args],
            shell=False,
            stderr=STDOUT,
            text=True,
        )
        m = re.search("runtime=([0-9\.]+)", out)
        assert m
        return {
            "runtime": float(m.group(1)),
        }

    # run the measurements `repeat` times, to get a basic uncertainty estimate
    data = [run() for _ in range(repeat)]
    return {k: np.array([d[k] for d in data]) for k in data[0].keys()}


# Try various powers of 2 from 1 to 64
thread_counts = [1, 2, 4, 8, 16, 32, 64]
# Save performance data for each thread count separately
thread_data = {nt: ringtest("-nt", nt) for nt in thread_counts}

Now we have gathered the simulation runtimes for different thread values, we can plot these:

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.xscale("log", base=2)
plt.xlabel("Thread count")
plt.ylabel("Simulation runtime [s]")
yvals, yerrs_low, yerrs_high = [], [], []
for nt in thread_counts:
    runtime_measurements = thread_data[nt]["runtime"]
    yvals.append(runtime_measurements.mean())
    yerrs_low.append(yvals[-1] - runtime_measurements.min())
    yerrs_high.append(runtime_measurements.max() - yvals[-1])

plt.errorbar(thread_counts, yvals, yerr=(yerrs_low, yerrs_high))
plt.show()